In [1]:
!pip install category_encoders

    100% |████████████████████████████████| 61kB 25.7MB/s ta 0:00:01
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [27]:
!pip install --upgrade pip

    100% |████████████████████████████████| 1.4MB 23.7MB/s ta 0:00:01
  Found existing installation: pip 10.0.1
    Uninstalling pip-10.0.1:
      Successfully uninstalled pip-10.0.1


In [118]:
import pandas as pd
import category_encoders as ce
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
import os
import urllib.request
import boto3
import json
from sklearn.preprocessing import StandardScaler

In [21]:
# def download(url):
#     filename= url.split("/")[-1]
#     if not os.path.exists(filename):
#         urllib.request.urlretrieve(url, filename)

In [105]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)



In [5]:
!pip install kaggle

    100% |████████████████████████████████| 61kB 28.7MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 27.4MB/s ta 0:00:01
    100% |████████████████████████████████| 81kB 27.8MB/s ta 0:00:01
  Running setup.py bdist_wheel for kaggle ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/ee/97/c5/87dcdc9434fe4e632ed5945e31a03703af229db178ef6a00e8
  Running setup.py bdist_wheel for python-slugify ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/16/7f/c3/6b0582283ad589d68a306da924a78c74546e010d8106b9b3a9
Successfully built kaggle python-slugify
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [25]:
# download('https://www.kaggle.com/c/ds2-tree-ensembles/download/sample_submission.csv')
# download('https://www.kaggle.com/c/ds2-tree-ensembles/download/test_features.csv')
# download('https://www.kaggle.com/c/ds2-tree-ensembles/download/train_features.csv')
# download('https://www.kaggle.com/c/ds2-tree-ensembles/download/train_labels.csv')

In [35]:
KAGGLE_COMPETITION = 'ds2-tree-ensembles'

In [38]:
config = json.loads(open("kaggle.json").read())
os.environ['KAGGLE_USERNAME'] = config['username']
os.environ['KAGGLE_KEY'] = config['key']

In [39]:
!kaggle competitions download -c ds2-tree-ensembles

  0%|                                                | 0.00/313k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 313k/313k [00:00<00:00, 51.7MB/s]
  0%|                                               | 0.00/3.57M [00:00<?, ?B/s]
100%|███████████████████████████████████████| 3.57M/3.57M [00:00<00:00, 212MB/s]
 96%|███████████████████████████████████████▏ | 167M/174M [00:00<00:00, 168MB/s]
100%|█████████████████████████████████████████| 174M/174M [00:00<00:00, 186MB/s]
  0%|                                               | 0.00/4.86M [00:00<?, ?B/s]
100%|██████████████████████████████████████| 4.86M/4.86M [00:00<00:00, 73.7MB/s]


In [6]:
!mkdir -p data

In [7]:
!curl https://resources.lendingclub.com/LCDataDictionary.xlsx > ./data/data-description.xlsx

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 23582    0 23582    0     0  52521      0 --:--:-- --:--:-- --:--:-- 52521


In [40]:
!unzip test_features.csv.zip

Archive:  test_features.csv.zip
  inflating: test_features.csv       


In [41]:
!unzip train_features.csv.zip

Archive:  train_features.csv.zip
  inflating: train_features.csv      


In [42]:
!unzip train_labels.csv.zip

Archive:  train_labels.csv.zip
  inflating: train_labels.csv        


In [51]:
#change permissions

!chmod +r 'test_features.csv'
!chmod +r 'train_features.csv'
!chmod +r 'train_labels.csv'




In [52]:
train_df = pd.read_csv('train_features.csv')

In [53]:
train_df.shape

(1309457, 103)

In [54]:
test_df = pd.read_csv('test_features.csv')

In [55]:
test_df.shape

(26724, 103)

In [56]:
sample = pd.read_csv('sample_submission.csv')


In [57]:
sample.shape

(26724, 2)

In [58]:
train_labels_df = pd.read_csv('train_labels.csv')
train_labels_df.shape

(1309457, 2)

In [59]:
train_new = pd.merge(train_df,train_labels_df,how='inner',on='id')

In [60]:
train_new.shape

(1309457, 104)

In [73]:
train_new_sample = train_new.sample(frac=0.1, replace=True, random_state=1)

In [165]:
train_new_sample.shape

(130946, 104)

In [179]:
y_train =  train_new_sample['charged_off']
X_train = train_new_sample.drop(columns=['charged_off','mths_since_last_delinq',
'open_acc_6m',
'open_act_il',
'open_il_12m',
'open_il_24m',
'mths_since_rcnt_il',
'total_bal_il',
'il_util',
'open_rv_12m',
'open_rv_24m',
'max_bal_bc',
'all_util',
'mths_since_recent_bc_dlq',
'mths_since_recent_revol_delinq',
'inq_fi',
'total_cu_tl',
'inq_last_12m',
'mths_since_last_major_derog',
'annual_inc_joint',
'dti_joint',
'member_id',
'url',
'desc',
'mths_since_last_record',
'revol_bal_joint',
'sec_app_earliest_cr_line',
'sec_app_inq_last_6mths',
'sec_app_mort_acc',
'sec_app_open_acc',
'sec_app_revol_util',
'sec_app_open_act_il',
'sec_app_num_rev_accts',
'sec_app_chargeoff_within_12_mths',
'sec_app_collections_12_mths_ex_med',
'sec_app_mths_since_last_major_derog',
# 'term',
'int_rate',
# 'grade',
'sub_grade',
'emp_title',
'emp_length',
# 'home_ownership',
'purpose',
'title',
'zip_code',
'addr_state',
'earliest_cr_line',
'revol_util',
'initial_list_status',
'application_type',
'disbursement_method'
])

X_test  = test_df.drop(columns=['mths_since_last_delinq',
'open_acc_6m',
'open_act_il',
'open_il_12m',
'open_il_24m',
'mths_since_rcnt_il',
'total_bal_il',
'il_util',
'open_rv_12m',
'open_rv_24m',
'max_bal_bc',
'all_util',
'mths_since_recent_bc_dlq',
'mths_since_recent_revol_delinq',
'inq_fi',
'total_cu_tl',
'inq_last_12m',
'mths_since_last_major_derog',
'annual_inc_joint',
'dti_joint',
'member_id',
'url',
'desc',
'mths_since_last_record',
'revol_bal_joint',
'sec_app_earliest_cr_line',
'sec_app_inq_last_6mths',
'sec_app_mort_acc',
'sec_app_open_acc',
'sec_app_revol_util',
'sec_app_open_act_il',
'sec_app_num_rev_accts',
'sec_app_chargeoff_within_12_mths',
'sec_app_collections_12_mths_ex_med',
'sec_app_mths_since_last_major_derog',
# 'term',
'int_rate',
# 'grade',
'sub_grade',
'emp_title',
'emp_length',
# 'home_ownership',
'purpose',
'title',
'zip_code',
'addr_state',
'earliest_cr_line',
'revol_util',
'initial_list_status',
'application_type',
'disbursement_method'
])

In [180]:
#fill in NaN
X_train = X_train.fillna(X_train.median())
X_test = X_test.fillna(X_test.median())


In [181]:
X_train.isnull().sum()

id                            0
loan_amnt                     0
funded_amnt                   0
term                          0
installment                   0
grade                         0
home_ownership                0
annual_inc                    0
dti                           0
delinq_2yrs                   0
inq_last_6mths                0
open_acc                      0
pub_rec                       0
revol_bal                     0
total_acc                     0
collections_12_mths_ex_med    0
acc_now_delinq                0
tot_coll_amt                  0
tot_cur_bal                   0
total_rev_hi_lim              0
acc_open_past_24mths          0
avg_cur_bal                   0
bc_open_to_buy                0
bc_util                       0
chargeoff_within_12_mths      0
delinq_amnt                   0
mo_sin_old_il_acct            0
mo_sin_old_rev_tl_op          0
mo_sin_rcnt_rev_tl_op         0
mo_sin_rcnt_tl                0
mort_acc                      0
mths_sin

In [182]:
X_train.isnull().sum()

id                            0
loan_amnt                     0
funded_amnt                   0
term                          0
installment                   0
grade                         0
home_ownership                0
annual_inc                    0
dti                           0
delinq_2yrs                   0
inq_last_6mths                0
open_acc                      0
pub_rec                       0
revol_bal                     0
total_acc                     0
collections_12_mths_ex_med    0
acc_now_delinq                0
tot_coll_amt                  0
tot_cur_bal                   0
total_rev_hi_lim              0
acc_open_past_24mths          0
avg_cur_bal                   0
bc_open_to_buy                0
bc_util                       0
chargeoff_within_12_mths      0
delinq_amnt                   0
mo_sin_old_il_acct            0
mo_sin_old_rev_tl_op          0
mo_sin_rcnt_rev_tl_op         0
mo_sin_rcnt_tl                0
mort_acc                      0
mths_sin

In [114]:
X_test.isnull().sum()

id                            0
loan_amnt                     0
funded_amnt                   0
installment                   0
annual_inc                    0
dti                           0
delinq_2yrs                   0
inq_last_6mths                0
open_acc                      0
pub_rec                       0
revol_bal                     0
total_acc                     0
collections_12_mths_ex_med    0
acc_now_delinq                0
tot_coll_amt                  0
tot_cur_bal                   0
total_rev_hi_lim              0
acc_open_past_24mths          0
avg_cur_bal                   0
bc_open_to_buy                0
bc_util                       0
chargeoff_within_12_mths      0
delinq_amnt                   0
mo_sin_old_il_acct            0
mo_sin_old_rev_tl_op          0
mo_sin_rcnt_rev_tl_op         0
mo_sin_rcnt_tl                0
mort_acc                      0
mths_since_recent_bc          0
mths_since_recent_inq         0
num_accts_ever_120_pd         0
num_actv

In [170]:
X_train.dtypes

id                              int64
loan_amnt                     float64
funded_amnt                   float64
term                           object
int_rate                       object
installment                   float64
grade                          object
sub_grade                      object
emp_title                      object
emp_length                     object
home_ownership                 object
annual_inc                    float64
purpose                        object
title                          object
zip_code                       object
addr_state                     object
dti                           float64
delinq_2yrs                   float64
earliest_cr_line               object
inq_last_6mths                float64
open_acc                      float64
pub_rec                       float64
revol_bal                     float64
revol_util                     object
total_acc                     float64
initial_list_status            object
collections_

In [116]:
X_test.dtypes

id                              int64
loan_amnt                     float64
funded_amnt                   float64
installment                   float64
annual_inc                    float64
dti                           float64
delinq_2yrs                   float64
inq_last_6mths                float64
open_acc                      float64
pub_rec                       float64
revol_bal                     float64
total_acc                     float64
collections_12_mths_ex_med    float64
acc_now_delinq                float64
tot_coll_amt                  float64
tot_cur_bal                   float64
total_rev_hi_lim              float64
acc_open_past_24mths          float64
avg_cur_bal                   float64
bc_open_to_buy                float64
bc_util                       float64
chargeoff_within_12_mths      float64
delinq_amnt                   float64
mo_sin_old_il_acct            float64
mo_sin_old_rev_tl_op          float64
mo_sin_rcnt_rev_tl_op         float64
mo_sin_rcnt_

In [90]:
X_train.head()

,id,loan_amnt,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,disbursement_method
128037,2002225,1800.0,1800.0,36 months,15.61%,62.94,C,C3,plant foreman,10+ years,MORTGAGE,57000.0,home_improvement,Home improvement,054xx,VT,28.44,0.0,Oct-2001,0.0,11.0,0.0,10948.0,97.7%,26.0,f,0.0,Individual,0.0,0.0,119594.0,11200.0,7.0,13288.0,248.0,97.6,0.0,0.0,145.0,100.0,16.0,7.0,2.0,23.0,7.0,0.0,3.0,4.0,4.0,6.0,14.0,6.0,10.0,4.0,11.0,0.0,0.0,0.0,2.0,100.0,100.0,0.0,0.0,141147.0,69233.0,10500.0,65447.0,Cash
491755,2006651,4550.0,4550.0,36 months,15.61%,159.09,D,D1,Office Professional,3 years,RENT,22500.0,debt_consolidation,Debt consolidation,900xx,CA,28.32,0.0,Sep-2008,0.0,15.0,0.0,4232.0,20.3%,27.0,f,0.0,Individual,0.0,0.0,20815.0,20800.0,6.0,1487.0,143.0,92.5,0.0,0.0,73.0,51.0,7.0,7.0,0.0,22.0,7.0,0.0,2.0,4.0,2.0,2.0,15.0,10.0,12.0,4.0,15.0,0.0,0.0,0.0,3.0,100.0,50.0,0.0,0.0,40520.0,20815.0,1900.0,19720.0,Cash
470924,1917915,14975.0,14975.0,60 months,17.57%,376.78,D,D4,sales rep,10+ years,MORTGAGE,75000.0,debt_consolidation,Debt consolidation,148xx,NY,22.58,0.0,Sep-1995,0.0,9.0,0.0,2082.0,33%,25.0,w,0.0,Individual,0.0,3682.0,41699.0,6300.0,1.0,4633.0,162.0,89.2,0.0,0.0,157.0,224.0,25.0,21.0,2.0,25.0,14.0,2.0,2.0,6.0,2.0,3.0,11.0,7.0,12.0,6.0,9.0,0.0,0.0,0.0,0.0,87.5,100.0,0.0,0.0,71428.0,41699.0,1500.0,65128.0,Cash
491263,1665617,5350.0,5350.0,36 months,18.55%,194.90,E,E2,Certified Nursing Assistance,< 1 year,RENT,31000.0,home_improvement,Home improvement,282xx,NC,18.35,0.0,Aug-2005,0.0,8.0,1.0,1587.0,79.3%,13.0,f,1.0,Individual,0.0,697.0,27805.0,2000.0,0.0,3476.0,180.0,89.4,0.0,0.0,56.0,120.0,25.0,25.0,0.0,25.0,1.0,1.0,2.0,3.0,2.0,4.0,6.0,3.0,7.0,3.0,8.0,0.0,0.0,0.0,0.0,76.9,100.0,0.0,0.0,34145.0,27805.0,1700.0,32145.0,Cash
836489,1383505,4900.0,4900.0,60 months,7.88%,99.08,A,A5,Mifflin Township Division of Fire,6 years,MORTGAGE,63600.0,car,Motorcycle,431xx,OH,14.00,0.0,Jun-1997,0.0,13.0,0.0,38566.0,15.4%,33.0,f,0.0,Individual,0.0,0.0,81108.0,24000.0,4.0,7491.5,4655.0,63.5,0.0,0.0,129.0,164.0,8.0,5.0,1.0,13.0,5.0,0.0,3.0,5.0,4.0,7.0,7.0,7.0,13.0,5.0,11.0,0.0,0.0,0.0,2.0,97.9,42.9,0.0,0.0,113960.0,37220.0,15000.0,31644.0,Cash


In [142]:
X_train['annual_inc'].unique()

array([ 57000.,  22500.,  75000., ...,  56803.,  58275., 114297.])

In [171]:
columns = list(X_train) 

unique_values =[]

for i in columns:
    unique_values.append(X_train[i].nunique())

In [172]:
Unique = pd.DataFrame(unique_values)


In [173]:
Unique['Category'] = list(X_train)

In [174]:
#keep categories under 10
Unique

,0,Category
0,124530,id
1,1401,loan_amnt
2,1401,funded_amnt
3,2,term
4,577,int_rate
5,32541,installment
6,7,grade
7,35,sub_grade
8,53465,emp_title
9,11,emp_length


In [151]:
list(X_train)

['id',
 'loan_amnt',
 'funded_amnt',
 'installment',
 'annual_inc',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'total_acc',
 'collections_12_mths_ex_med',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'total_rev_hi_lim',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'bc_open_to_buy',
 'bc_util',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
 'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 'mort_acc',
 'mths_since_recent_bc',
 'mths_since_recent_inq',
 'num_accts_ever_120_pd',
 'num_actv_bc_tl',
 'num_actv_rev_tl',
 'num_bc_sats',
 'num_bc_tl',
 'num_il_tl',
 'num_op_rev_tl',
 'num_rev_accts',
 'num_rev_tl_bal_gt_0',
 'num_sats',
 'num_tl_120dpd_2m',
 'num_tl_30dpd',
 'num_tl_90g_dpd_24m',
 'num_tl_op_past_12m',
 'pct_tl_nvr_dlq',
 'percent_bc_gt_75',
 'pub_rec_bankruptcies',
 'tax_liens',
 'tot_hi_cred_lim',
 'total_bal_ex_mort',
 'total_bc_limit',
 'total_il_high_credit_limit']

In [108]:
y_train.shape

(130946,)

In [109]:
X_train.shape

(130946, 52)

In [110]:
X_test.shape

(26724, 52)

In [184]:
pipe = make_pipeline(ce.OneHotEncoder(use_cat_names=True),
    StandardScaler(), 
    DecisionTreeClassifier(max_depth=10)
)

# cross_val_score(pipe, X_train, y_train, cv=5, scoring='roc_auc', n_jobs=-1)

pipe.fit(X_train, y_train)



Pipeline(memory=None,
     steps=[('onehotencoder', OneHotEncoder(cols=['term', 'grade', 'home_ownership'], drop_invariant=False,
       handle_unknown='impute', impute_missing=True, return_df=True,
       use_cat_names=True, verbose=0)), ('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('decision...      min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))])

In [185]:
y_pred = pipe.predict(X_test)


In [186]:
y_pred = pd.DataFrame(y_pred)

y_pred[0].value_counts()

0    25342
1     1382
Name: 0, dtype: int64

In [187]:
sample_submission = pd.read_csv('sample_submission.csv') 


In [188]:
submission = sample_submission.copy() 


In [189]:
submission['charged_off'] = pipe.predict_proba(X_test)[:, 1] 

In [190]:
submission['charged_off'].head()

0    0.109850
1    0.147704
2    0.109850
3    0.210790
4    0.037586
Name: charged_off, dtype: float64

In [191]:
submission.to_csv('submission-002.csv', index=False)